In [10]:

!pip install pdfplumber transformers sentence-transformers pytesseract faiss-cpu opencv-python-headless Pillow

import pdfplumber
import pytesseract
from PIL import Image
import cv2
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import faiss
from sentence_transformers import util

# Initialize necessary models
nlp_model = SentenceTransformer('all-MiniLM-L6-v2')
cross_encoder_model = BertForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
cross_encoder_tokenizer = BertTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
index = faiss.IndexFlatL2(384)  # FAISS index for vector search

# Mock chat memory (to replace Redis)
chat_memory = {}

def store_chat_memory(session_id, question, answer):
    chat_memory[session_id] = {"question": question, "answer": answer}

def retrieve_chat_memory(session_id):
    return chat_memory.get(session_id)

# 1. PDF Text Extraction using pdfplumber
def extract_text_from_pdf(pdf_path, is_scanned=False, lang='eng'):
    if not is_scanned:
        # Use pdfplumber for extracting text from digital PDFs
        text = ""
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text()
        return text
    else:
        # OCR extraction for scanned PDFs
        img = cv2.imread(pdf_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return pytesseract.image_to_string(gray, lang=lang)

# 2. Query decomposition
def decompose_query(query):
    sub_queries = query.split(' and ')  # Simple example, can be enhanced with NLP models
    return sub_queries

# 3. Optimized chunking for large documents
def chunk_text(text, max_tokens=200):
    chunks = []
    chunk = ""
    for sentence in text.split('. '):
        if len(chunk.split()) + len(sentence.split()) < max_tokens:
            chunk += sentence + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + ". "
    chunks.append(chunk.strip())
    return chunks


def hybrid_search(query):

    keyword_results = [{'text': 'This is a mocked keyword search result.'}]

    # Semantic Search using SentenceTransformer
    query_embedding = nlp_model.encode(query)
    sentence_embeddings = nlp_model.encode([result['text'] for result in keyword_results])
    semantic_results = util.semantic_search(query_embedding, sentence_embeddings)

    return keyword_results, semantic_results

# 5. FAISS vector search
def add_to_faiss_index(embeddings):
    index.add(embeddings)

def search_faiss(query_embedding, top_k=10):
    distances, indices = index.search(np.array([query_embedding]), top_k)
    return distances, indices

# 6. Reranking using Cross-Encoder
def rerank(query, documents):
    reranked_results = []
    for doc in documents:
        inputs = cross_encoder_tokenizer(query, doc, return_tensors="pt", padding=True, truncation=True)
        scores = cross_encoder_model(**inputs).logits
        reranked_results.append((doc, scores.item()))

    reranked_results.sort(key=lambda x: x[1], reverse=True)
    return reranked_results

# RAG pipeline
def rag_pipeline(pdf_path, is_scanned=False, lang='eng', session_id=None, query=None):
    # 1. Extract text
    text = extract_text_from_pdf(pdf_path, is_scanned, lang)

    # 2. Store extracted text in chunks
    chunks = chunk_text(text)

    # 3. Add chunk embeddings to FAISS
    embeddings = nlp_model.encode(chunks)
    add_to_faiss_index(embeddings)

    # 4. Perform Hybrid Search (Keyword + Semantic)
    keyword_results, semantic_results = hybrid_search(query)

    # 5. Perform FAISS search for the query embedding
    query_embedding = nlp_model.encode(query)
    distances, indices = search_faiss(query_embedding)

    # 6. Rerank the results using the cross-encoder model
    reranked = rerank(query, chunks)

    # 7. Store the conversation context in chat memory
    if session_id:
        store_chat_memory(session_id, query, reranked[0][0])

    return {
        "keyword_results": keyword_results,
        "semantic_results": semantic_results,
        "faiss_results": indices,
        "reranked_results": reranked
    }





/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
from google.colab import files
uploaded = files.upload()



Saving Reboot_Leadership_and_the_Art_of.pdf to Reboot_Leadership_and_the_Art_of.pdf
{'keyword_results': [{'text': 'This is a mocked keyword search result.'}], 'semantic_results': [[{'corpus_id': 0, 'score': 0.05150983855128288}]], 'faiss_results': array([[145,  42, 203,  24,  18,  49, 215, 307, 367,  55]]), 'reranked_results': [('Or that when a stranger attacks us on the\nsubway, knocks us unconscious, with blood gushing from our nose and a\nbroken tooth lying next to our dazed head, we can awaken to marvel at the\ncaring efficiency of a young New York City police officer as she presses the\nspeed-dial button on your phone, the one marked, “Home.” Or, amid the\nblood and broken bone, we feel the wise hands of an even younger\nemergency room nurse at Bellevue Hospital—his hands gently cradling\nyour head as he asks if you know what year it is and the name of the\npresident.\nWhat would it be like if we were to stop externalizing responsibility for\nour inner state onto the Irrational Ot

In [11]:
pdf_path = next(iter(uploaded.keys()))  # Get the first uploaded file
session_id = "user_123"
query = "who is Jerry?"

# Run the RAG pipeline
results = rag_pipeline(pdf_path, is_scanned=False, lang='eng', session_id=session_id, query=query)

# Display the results
print(results)

{'keyword_results': [{'text': 'This is a mocked keyword search result.'}], 'semantic_results': [[{'corpus_id': 0, 'score': 0.08191931992769241}]], 'faiss_results': array([[380, 379,   4, 382, 375,  71,   3,  41, 312,   0]]), 'reranked_results': [('In his\nlife and work—and in this book—Jerry is a remarkable combination\nof fierce and gentle, gravitas and humor, laserlike intellect and\nplayful imagination. He knows that the unexamined life is not worth\nliving. He also knows that if you choose to live such a life, you\nshould never, ever take a job that involves other people. Jerry’s way\nof working with leaders has nothing to do with ‘seven steps to\nsuccess’ or posturing as an advocate for high-minded leadership\nvirtues. Instead, he helps us untangle the inner knots with which we\nhobble first ourselves, then our colleagues, then the work we do,\nthen the world. In these beautifully written pages—full of engaging\nstories, bedrock truths, and practical tools—you’ll learn about\nJerr